In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# otimizador Adam
from tensorflow.keras.optimizers import Adam

# O padrão é 0.001. Vamos tentar um valor menor.
optimizer = Adam(learning_rate=0.0005)

# --- Parâmetros ---
IMAGE_WIDTH = 150
IMAGE_HEIGHT = 150
BATCH_SIZE = 32
EPOCHS = 25

# --- Diretórios do Dataset ---
# IMPORTANT: Update the path below to where your 'chest_xray' folder is located in your Google Drive.
# After mounting your Google Drive, it will typically be under /content/drive/MyDrive/
# base_dir_drive = '/content/chest_xray/' # <--- **UPDATE THIS PATH**
base_dir_drive = '/kaggle/input/chest-xray-pneumonia/chest_xray/' # <--- **UPDATE THIS PATH**

train_dir = os.path.join(base_dir_drive, 'train')
validation_dir = os.path.join(base_dir_drive, 'val')
test_dir = os.path.join(base_dir_drive, 'test')


# --- Check if directories exist ---
print(f"Current working directory: {os.getcwd()}")
print(f"Checking for dataset in: {base_dir_drive}")

if not os.path.exists(train_dir):
    print(f"Error: Training directory not found at {train_dir}")
    print("Please make sure the 'chest_xray' folder is in the correct location in your Google Drive and the 'base_dir_drive' path above is correct.")
if not os.path.exists(validation_dir):
    print(f"Error: Validation directory not found at {validation_dir}")
if not os.path.exists(test_dir):
    print(f"Error: Test directory not found at {test_dir}")


# --- Geradores de Dados com Data Augmentation ---
# Ensure directories exist before creating generators
if os.path.exists(train_dir) and os.path.exists(validation_dir) and os.path.exists(test_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # O gerador de validação e teste não deve ter data augmentation, apenas o rescale
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    # --- Construção do Modelo ---
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dropout(0.5), # Dropout para regularização

        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid') # Sigmoid para classificação binária
    ])

    # --- Compilação do Modelo ---
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # --- Treinamento do Modelo ---
    # Only attempt to train if generators were created successfully
    if train_generator.samples > 0 and validation_generator.samples > 0:
        history = model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // BATCH_SIZE
        )

        # --- Avaliação do Modelo ---
        # Only attempt to evaluate if test generator has samples
        if test_generator.samples > 0:
            test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
            print(f'Acurácia no conjunto de teste: {test_acc:.2f}')
        else:
            print("Skipping model evaluation: No images found in the test directory.")
    else:
        print("Skipping model training and evaluation: No images found in training or validation directories.")

else:
    print("Skipping data generation, model training, and evaluation due to missing directories.")

In [ ]:
# Salva o modelo inteiro em um único arquivo .h5
model.save('detector_pneumonia.h5')

print("Modelo salvo com sucesso!")